In [ ]:
!pip3 install scikit-learn==1.0 -U
!pip install imblearn
!pip install pymorphy2
!pip install optuna

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.metrics import make_scorer

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.neural_network import MLPClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost as xgb

from tqdm import tqdm

from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.ensemble import EasyEnsembleClassifier

from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTEN
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTEN

from imblearn.over_sampling import RandomOverSampler 

from imblearn.under_sampling import RandomUnderSampler

from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NeighbourhoodCleaningRule

from sklearn.preprocessing import normalize
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

import pymorphy2
import optuna

import lightgbm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def sort(arr, n):
    for i in range(1, n):
        temp = arr[i] 
        j = i - 1
        while j >= 0 and len(temp) > len(arr[j]):
            arr[j + 1] = arr[j]
            j -= 1
 
        arr[j + 1] = temp

Откроем файл с данными.

In [ ]:
df = pd.read_csv('credit_train.csv', index_col='id')

In [ ]:
df.head()

,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income,opened
id,,,,,,,,,,,,,,
26031,M,24,UNM,"24000,00",10,1.60,SCH,2.0,0.0,SPC,"0,728851",ОБЛ ЛЕНИНГРАДСКАЯ,35000.0,True
148877,M,23,UNM,"16339,00",18,1.10,SCH,2.0,0.0,SPC,"0,347385",МОСКВА Г,38000.0,True
142019,M,54,UNM,"15652,00",10,1.32,GRD,2.0,0.0,SPC,"0,349705",ОБЛ МОСКОВСКАЯ,38000.0,True
146816,F,52,UNM,"16560,00",10,1.10,SCH,1.0,0.0,SPC,"0,359232",РЕСП МОРДОВИЯ,28000.0,False
27699,F,63,WID,"9078,00",18,1.10,SCH,4.0,0.0,SPC,"0,529368",ЧЕЛЯБИНСКАЯ ОБЛ,30000.0,False


Удалим колонки с которыми мы не умеем работать (ну или вы умеете :D)

In [ ]:
from pandas.core.arrays import categorical
df = pd.read_csv('credit_train.csv')

df = df.drop_duplicates()
df = df.fillna('-1')

# df = df.dropna()
df['region'] = df['region'].apply(lambda x: str(x))
y_train = df['opened']
X_train = df.drop(columns=['opened', 'id'])
X_train['credit_sum'] = X_train['credit_sum'].apply(lambda x: int(float(x.replace(',', '.'))))
X_train['credit_score'] = X_train['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_train['credit_score'] = X_train['credit_score'].apply(lambda x: float("{:.2f}".format(x)))
X_train['credit_count'] = X_train['credit_count'].apply(lambda x: int(x))
X_train['overdue_credit_count'] = X_train['overdue_credit_count'].apply(lambda x: int(x))
X_train['income'] = X_train['income'].apply(lambda x: int(x))

# X_train['gender'] = np.array(X_train['gender'])
# X_train['gender'] = X_train['gender'].values.reshape(-1, 1)
# onehotencoder1 = OrdinalEncoder()
# X_train['gender'] = onehotencoder1.fit_transform(X_train['gender']).toarray()



X_train['gender'] = X_train['gender'].apply(lambda x: int(x.replace('F','0').replace('M', '1')))
X_train['marital_status'] = X_train['marital_status'].apply(lambda x: int(x.replace('MAR','1').replace('UNM', '2').replace('DIV', '3').replace('CIV', '4').replace('WID', '5')))
X_train['job_pos'] = X_train['job_pos'].apply(lambda x: int(x.replace('SPC','1').replace('BIU', '2').replace('UMN', '3').replace('WOI', '4').replace('ATP', '5').replace('INP','6').replace('BIS', '7').replace('HSK', '8').replace('DIR', '9').replace('PNV', '10').replace('PNA','11').replace('PNI', '12').replace('WRP', '13').replace('PNS', '14').replace('WRK', '15').replace('ONB','16').replace('NOR', '17').replace('INV', '18')))
X_train['education'] = X_train['education'].apply(lambda x: int(x.replace('SCH','1').replace('GRD', '2').replace('UGR', '3').replace('PGR', '4').replace('ACD', '5')))
X_train['region'] = X_train['region'].apply(lambda x: str(x).replace('ОБЛАСТЬ','').replace('РЕСПУБЛИКА','').replace('ОБЛ','').replace('ОКРУГ','').replace('РЕСП','').replace('КРАЙ','').replace('Г','').replace('.','').replace('Р-Н','').replace(' ',''))




# X = word_tokenize(X_train['region'][0])

# morph = pymorphy2.MorphAnalyzer()

# w_lem_rus = [morph.normal_forms(word)[0] for word in X]

# n = len(w_lem_rus)
# sort(w_lem_rus, n)

# X_train['gender']

In [ ]:
X_train['region'].size

114399

In [ ]:
morph = pymorphy2.MorphAnalyzer()

for i in tqdm(range(X_train['region'].size)):
  w_lem = [morph.normal_forms(word)[0] for word in word_tokenize(X_train['region'][i])]
  n = len(w_lem)
  sort(w_lem, n)

  X_train['region'][i] = w_lem[0][0:4]

  0%|          | 0/114399 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 114399/114399 [03:52<00:00, 491.67it/s]


In [ ]:
print(X_train['region'].value_counts()[0:60])


In [ ]:
diction = {}
i = 1
for reg in X_train.region.unique():
    diction[reg] = i
    i += 1

diction
# X_train['region'] = 0

# # X_train['region'][148877]

for id in tqdm(X_train.index):
    X_train['region'][id] = diction[X_train['region'][id]]


  0%|          | 0/114399 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
100%|██████████| 114399/114399 [01:40<00:00, 1133.35it/s]


In [ ]:
X_train.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            114389, 114390, 114391, 114392, 114393, 114394, 114395, 114396,
            114397, 114398],
           dtype='int64', length=114399)

In [ ]:
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X_train, y_train)

In [ ]:
X_resampled

In [ ]:

X_train.loc[((X_train.age >= 18) & (X_train.age <= 20)),  'age'] = 1
X_train.loc[((X_train.age >= 21) & (X_train.age <= 23)),  'age'] = 2
X_train.loc[((X_train.age >= 24) & (X_train.age <= 26)),  'age'] = 3
X_train.loc[((X_train.age >= 27) & (X_train.age <= 30)),  'age'] = 4
X_train.loc[((X_train.age >= 31) & (X_train.age <= 34)),  'age'] = 5
X_train.loc[((X_train.age >= 35) & (X_train.age <= 37)),  'age'] = 6
X_train.loc[((X_train.age >= 38) & (X_train.age <= 42)),  'age'] = 7
X_train.loc[((X_train.age >= 43) & (X_train.age <= 44)),  'age'] = 8
X_train.loc[((X_train.age >= 45) & (X_train.age <= 46)),  'age'] = 9
X_train.loc[((X_train.age >= 47) & (X_train.age <= 50)),  'age'] = 10
X_train.loc[((X_train.age >= 51) & (X_train.age <= 52)),  'age'] = 11
X_train.loc[((X_train.age >= 53) & (X_train.age <= 54)),  'age'] = 12
X_train.loc[((X_train.age >= 55) & (X_train.age <= 56)),  'age'] = 13
X_train.loc[((X_train.age >= 57) & (X_train.age <= 59)),  'age'] = 14
X_train.loc[((X_train.age >= 60) & (X_train.age <= 63)),  'age'] = 15
X_train.loc[((X_train.age >= 64) & (X_train.age <= 66)),  'age'] = 16
X_train.loc[((X_train.age >= 67) & (X_train.age <= 70)),  'age'] = 17
X_train.loc[((X_train.age >= 71) & (X_train.age <= 71)),  'age'] = 18



In [ ]:
scaler = StandardScaler().fit(X_train)
# scaler
X_scaled = scaler.transform(X_train)
X_scaled

array([[ 1.04273984, -0.99907137,  0.37696356, ...,  2.0969627 ,
        -1.19643963, -0.20434392],
       [ 1.04273984, -1.29760282,  0.37696356, ..., -0.95959044,
        -1.14555996, -0.08479663],
       [ 1.04273984,  1.68771168,  0.37696356, ..., -0.95959044,
        -1.14555996, -0.08479663],
       ...,
       [-0.95901198,  0.79211733, -0.73006389, ..., -1.12046166,
         2.16161847, -0.38764976],
       [-0.95901198,  1.09064878, -0.73006389, ...,  0.00563687,
         2.51777614, -0.96147675],
       [ 1.04273984,  2.28477458, -0.73006389, ..., -0.87915483,
         2.31425747, -0.4035894 ]])

In [ ]:
rus = RandomUnderSampler()

X_res, y_res = rus.fit_resample(X_scaled, y_train)

In [ ]:
sampler = BorderlineSMOTE()

X_res, y_res = sampler.fit_resample(X_scaled, y_train)

In [ ]:
X_res

array([[-0.12554381, -0.27819561,  1.17137564, ...,  0.03516984,
         2.0889326 , -0.20421823],
       [-0.5985378 ,  1.98720955, -0.95148058, ...,  0.03516984,
        -0.98080822, -0.08460703],
       [-0.64095353, -0.27819561, -0.01742384, ...,  0.03516984,
        -0.96213867, -0.08460703],
       ...,
       [ 2.25686816,  0.28815568, -0.95148058, ...,  0.03516984,
        -0.64760343,  2.38735765],
       [-1.09413895, -0.27819561,  1.17137564, ...,  0.03516984,
         1.01416413, -0.82264378],
       [ 0.6184845 ,  0.28815568,  1.31853099, ...,  0.03516984,
         2.34804364, -0.46473238]])

In [ ]:
# pt = PowerTransformer(method='box-cox', standardize=False)
# X_lognormal = pt.fit_transform(X_train)

X_normilased = normalize(X_train, norm='l2')


In [ ]:
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[0.10779463, 0.21212121, 0.625     , ..., 0.25      , 0.64597786,
        0.03174603],
       [0.06895835, 0.45454545, 0.10416667, ..., 0.25      , 0.307886  ,
        0.03492063],
       [0.06547571, 0.21212121, 0.33333333, ..., 0.25      , 0.3099422 ,
        0.03492063],
       ...,
       [0.06935883, 0.21212121, 0.10416667, ..., 0.25      , 0.29015564,
        0.02687831],
       [0.04781917, 0.21212121, 0.41666667, ..., 0.25      , 0.4142504 ,
        0.01164021],
       [0.0296202 , 0.21212121, 0.10416667, ..., 0.25      , 0.31489128,
        0.02645503]])

In [ ]:
X_lognormal

In [ ]:
X_normilased

array([[2.35636807e-05, 7.06910420e-05, 4.71273613e-05, ...,
        1.71744122e-05, 2.35636807e-05, 8.24728823e-01],
       [2.41757334e-05, 4.83514668e-05, 4.83514668e-05, ...,
        8.39828714e-06, 4.83514668e-05, 9.18677868e-01],
       [2.43325181e-05, 2.91990218e-04, 4.86650363e-05, ...,
        8.50920326e-06, 7.29975544e-05, 9.24635689e-01],
       ...,
       [0.00000000e+00, 2.60490556e-04, 2.89433952e-05, ...,
        9.47548871e-06, 2.08392445e-03, 8.79879213e-01],
       [0.00000000e+00, 4.97462370e-04, 4.97462370e-05, ...,
        2.32511424e-05, 4.02944520e-03, 7.95939792e-01],
       [3.20485622e-05, 4.48679871e-04, 3.20485622e-05, ...,
        1.13865016e-05, 2.40364217e-03, 9.61456866e-01]])

Обучим что-нибудь классное

In [ ]:
y_n = []
for i in range(len(y_train)):
  if y_train[i] == True:
    y_n.append(1)
  else:
    y_n.append(0)

In [ ]:
def objective(trial, data = X_scaled, target = y_train):
  train_x, test_x, train_y, test_y = train_test_split(X_scaled, y_n, test_size=0.15, random_state = 42)
  params = {
      'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 10.0),
      'reg_lambda': trial.suggest_float('reg_lanbda', 0.001, 10.0),
      'num_leaves': trial.suggest_int('num_leaves', 11, 333),
      'min_child_samples': trial.suggest_int('min_child_samples', 5, 108),
      'max_depth': trial.suggest_int('max_depth', 5, 64),
      'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.02, 0.05, 0.005, 0.11]),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
      'n_estimators': trial.suggest_int('n_estimators', 2000, 8000),
      'cat_smooth' : trial.suggest_int('cat_smooth', 10, 100),
      'cat_l2': trial. suggest_int('cat_l2', 1, 20),
      'min_data_per_group': trial.suggest_int("min_data_per_group", 50, 200),
      'cat_feature' : [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 58, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68],
      'n_jobs': -1,
      'random_state': 42,
      'boosting_type': 'gbdt',
      'metric': 'f1',
      'class_weight': {0: 0.61, 1: 2.83},
      }

  model = lightgbm.LGBMClassifier(**params)
  model.fit(train_x, train_y, eval_set=[(test_x, test_y)], eval_metric='f1', verbose=True)

  preds = model.predict(test_x) 

  f_1 = f1_score(test_y, preds)

  return f_1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)


[I 2022-05-14 17:45:40,578] A new study created in memory with name: no-name-4b703e83-54e3-4ea8-b4dc-df714b896362
[I 2022-05-14 17:46:43,711] Trial 0 finished with value: 0.4410973641742873 and parameters: {'reg_alpha': 4.823314090241269, 'reg_lanbda': 4.04600770612389, 'num_leaves': 261, 'min_child_samples': 46, 'max_depth': 20, 'learning_rate': 0.02, 'colsample_bytree': 0.2614044318041986, 'n_estimators': 4107, 'cat_smooth': 21, 'cat_l2': 12, 'min_data_per_group': 171}. Best is trial 0 with value: 0.4410973641742873.
[I 2022-05-14 17:47:20,980] Trial 1 finished with value: 0.42753787022383 and parameters: {'reg_alpha': 2.706865220828238, 'reg_lanbda': 8.86835449634802, 'num_leaves': 306, 'min_child_samples': 45, 'max_depth': 51, 'learning_rate': 0.11, 'colsample_bytree': 0.24579413719815155, 'n_estimators': 4781, 'cat_smooth': 61, 'cat_l2': 9, 'min_data_per_group': 59}. Best is trial 0 with value: 0.4410973641742873.
[I 2022-05-14 17:49:17,398] Trial 2 finished with value: 0.41636840

In [ ]:
# clf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
# clf.fit(X_lognormal, y_train)
# print('BalancedRandomForestClassifier', cross_val_score(clf, X_lognormal, y_train, cv = 5, scoring='f1').mean())

# clf = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
#                                 sampling_strategy='auto',
#                                 replacement=False,
#                                 random_state=0)
# clf.fit(X_lognormal, y_train)
# print('BalancedBaggingClassifier', cross_val_score(clf, X_lognormal, y_train, cv = 5, scoring='f1').mean())

# clf = BalancedRandomForestClassifier(n_estimators=200, max_depth=50)

# parameters = {
#     'n_estimators': range(10, 50, 5),
#     # 'base_estimator': AdaBoostClassifier()
# }

# grid = GridSearchCV(clf, parameters, cv=5, scoring='f1')

# grid.fit(X_train, y_train)

# clf.fit(X_train, y_train)
# print(cross_val_score(clf, X_train, y_train, cv = 5, scoring='f1').mean())
# clf.fit(X_ros, y_ros)
# print('EasyEnsembleClassifier', cross_val_score(clf, X_train, y_train, cv = 5, scoring='f1').mean())

# clf = EasyEnsembleClassifier()
# clf.fit(X_normilased, y_train)
# print('EasyEnsembleClassifier (X_normilased)', cross_val_score(clf, X_normilased, y_train, cv = 5, scoring='f1').mean())

# clf = RUSBoostClassifier(n_estimators=200, algorithm='SAMME.R')
# clf.fit(X_train, y_train)
# print('RUSBoostClassifier', cross_val_score(clf, X_train, y_train, cv = 5, scoring='f1').mean())

# # DecisionTreeClassifier


# clf = DecisionTreeClassifier()
# clf.fit(X_train, y_train)
# print('RUSBoostClassifier', cross_val_score(clf, X_train, y_train, cv = 5, scoring='f1').mean())

# parameters = {'n_estimators': range(5, 16, 1)}

tl = NeighbourhoodCleaningRule(n_neighbors=6)
X_res, y_res = tl.fit_resample(X_scaled, y_train)
clf = BalancedBaggingClassifier(base_estimator=BalancedRandomForestClassifier(), n_estimators=17)
clf.fit(X_res, y_res)
print('BalancedBaggingClassifier (NeighbourhoodCleaningRule)', cross_val_score(clf, X_scaled, y_train, cv = 5, scoring='f1').mean())

#BalancedBaggingClassifier(base_estimator=BalancedRandomForestClassifier()) - best 0.44526


BalancedBaggingClassifier (NeighbourhoodCleaningRule) 0.4410074780886807


In [ ]:
param = {
    'reg_alpha': 5.741429176964243,
    'reg_lanbda': 8.356164943438623,
    'num_leaves': 164,
    'min_child_samples': 23,
    'max_depth': 48,
    'learning_rate': 0.005,
    'colsample_bytree': 0.27584240575625024,
    'n_estimators': 5217,
    'cat_smooth': 33,
    'cat_l2': 14,
    'min_data_per_group': 103
}

model = lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=164, max_depth=48, learning_rate=0.005, n_estimators=5217,
                                min_child_samples=23, colsample_bytree=0.27584240575625024, reg_alpha=5.741429176964243,
                                reg_lambda=8.356164943438623, random_state=42, cat_smooth=33, cat_l2 = 14, min_data_per_group= 103,
                                metric='f1', error_score='raise')

model.fit(X_scaled, y_n, eval_metric='f1', verbose=True)

print('BalancedBaggingClassifier (NeighbourhoodCleaningRule)', cross_val_score(model, X_scaled, y_n, cv = 5, scoring='f1').mean())



In [ ]:
X_scaled


array([[ 1.04273984, -0.99907137,  0.37696356, ...,  2.0969627 ,
        -1.19643963, -0.20434392],
       [ 1.04273984, -1.29760282,  0.37696356, ..., -0.95959044,
        -1.14555996, -0.08479663],
       [ 1.04273984,  1.68771168,  0.37696356, ..., -0.95959044,
        -1.14555996, -0.08479663],
       ...,
       [-0.95901198,  0.79211733, -0.73006389, ..., -1.12046166,
         2.16161847, -0.38764976],
       [-0.95901198,  1.09064878, -0.73006389, ...,  0.00563687,
         2.51777614, -0.96147675],
       [ 1.04273984,  2.28477458, -0.73006389, ..., -0.87915483,
         2.31425747, -0.4035894 ]])

In [ ]:
grid.best_score_

0.4424120885192556

In [ ]:
grid.best_params_

{'n_estimators': 15}

In [ ]:
y_pred = clf.predict(X_scaled)

In [ ]:
accuracy_score(y_train, y_pred)

0.6898195706571842

In [ ]:
cross_val_score(clf, X_train, y_train, cv = 5, scoring='f1').mean()

0.43493946851996024

In [ ]:
grid.best_score_

0.43814920671855184

In [ ]:
grid.best_params_

{'n_estimators': 20}

In [ ]:
clf = BalancedRandomForestClassifier()

parameters = {
    'n_estimators': range(10, 200, 10),
    'max_depth': range(2, 50, 2),
    'min_samples_split': range(2, 5, 1),
    'min_samples_leaf': range(1, 5, 1),
    'bootstrap' : [True, False],
    'oob_score' : [True, False],   

    # 'base_estimator': AdaBoostClassifier()
}

grid = GridSearchCV(clf, parameters, cv=5, scoring='f1')

grid.fit(X_train, y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
clf = EasyEnsembleClassifier()
clf.fit(X_train, y_train)
cross_val_score(clf, X_train, y_train, cv = 5, scoring='f1').mean()

0.437265956961142

Считаем тестовую выборку и предскажем ответы для нее

In [ ]:
df_test = pd.read_csv('credit_test.csv', index_col='id')
df_test.head()

,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income
id,,,,,,,,,,,,,
39162,F,18,UNM,"39469,00",6,1.44,GRD,NaN,NaN,SPC,"0,424628",САНКТ-ПЕТЕРБУРГ Г,30000.0
10673,F,63,WID,"16693,00",10,1.32,GRD,2.0,0.0,SPC,"0,311945",ОБЛ ЛЕНИНГРАДСКАЯ,21250.0
121409,M,42,MAR,"60260,00",10,1.10,GRD,5.0,0.0,SPC,"0,421982",МОСКВА Г,150000.0
23504,M,41,MAR,"32479,00",12,1.10,GRD,1.0,0.0,DIR,"0,406572",ТАТАРСТАН РЕСП,150000.0
69269,F,24,UNM,"8159,00",12,1.10,SCH,2.0,0.0,SPC,"0,418243",ТВЕРСКАЯ ОБЛ,16000.0


Удалим колонки с которыми мы не работаем, как и на тренировочной выборке

In [ ]:
# X_test = df_test.drop(columns=['gender', 'marital_status', 'education', 'job_pos', 'region'])
# X_test['credit_sum'] = X_test['credit_sum'].apply(lambda x: float(x.replace(',', '.')))
# X_test['credit_score'] = X_test['credit_score'].apply(lambda x: float(x.replace(',', '.')))
# X_test = X_test.fillna(-1)


# X_test = df_test.drop(columns=['region'])
X_test = df_test
X_test['credit_sum'] = X_test['credit_sum'].apply(lambda x: int(float(x.replace(',', '.'))))
X_test['credit_score'] = X_test['credit_score'].apply(lambda x: float(x.replace(',', '.')))
X_test['credit_score'] = X_test['credit_score'].apply(lambda x: float("{:.2f}".format(x)))
X_test = X_test.fillna('-1')
X_test['credit_count'] = X_test['credit_count'].apply(lambda x: int(x))
X_test['overdue_credit_count'] = X_test['overdue_credit_count'].apply(lambda x: int(x))
X_test['income'] = X_test['income'].apply(lambda x: int(x))
X_test['gender'] = X_test['gender'].apply(lambda x: int(x.replace('F','0').replace('M', '1')))
X_test['marital_status'] = X_test['marital_status'].apply(lambda x: int(x.replace('MAR','1').replace('UNM', '2').replace('DIV', '3').replace('CIV', '4').replace('WID', '5')))
X_test['job_pos'] = X_test['job_pos'].apply(lambda x: int(x.replace('SPC','1').replace('BIU', '2').replace('UMN', '3').replace('WOI', '4').replace('ATP', '5').replace('INP','6').replace('BIS', '7').replace('HSK', '8').replace('DIR', '9').replace('PNV', '10').replace('PNA','11').replace('PNI', '12').replace('WRP', '13').replace('PNS', '14').replace('WRK', '15').replace('ONB','16').replace('NOR', '17').replace('INV', '18')))
X_test['education'] = X_test['education'].apply(lambda x: int(x.replace('SCH','1').replace('GRD', '2').replace('UGR', '3').replace('PGR', '4').replace('ACD', '5')))
X_test['region'] = X_test['region'].apply(lambda x: str(x).replace('ОБЛАСТЬ','').replace('РЕСПУБЛИКА','').replace('ОБЛ','').replace('ОКРУГ','').replace('РЕСП','').replace('КРАЙ','').replace('Г','').replace('.','').replace('Р-Н','').replace(' ',''))

In [ ]:
df_test.region

id
39162     САНКТ-ПЕТЕРБУРГ Г
10673     ОБЛ ЛЕНИНГРАДСКАЯ
121409             МОСКВА Г
23504        ТАТАРСТАН РЕСП
69269          ТВЕРСКАЯ ОБЛ
                ...        
14148             САМАРСКАЯ
153732        САМАРСКАЯ ОБЛ
37647        РОСТОВСКАЯ ОБЛ
92808      ОБЛ СВЕРДЛОВСКАЯ
140311      ОБЛ ВОЛОГОДСКАЯ
Name: region, Length: 56347, dtype: object

In [ ]:
# diction = {}
diction_test = {}
i = 286
for reg in X_test.region.unique():
    diction_test[reg] = i
    i += 1

for key in diction:
    diction_test[key] = diction[key]

# diction_test['ДАЛЬНИЙВОСТОК']
# X_test['region'] = 0

# X_train['region'][148877]

for id in tqdm(X_test.index):
    X_test['region'][id] = diction_test[X_test['region'][id]]
diction

In [ ]:
X_train

,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,income
0,26031,1,24,2,24000,10,1.60,1,2,0,1,0.728851,35000
1,148877,1,23,2,16339,18,1.10,1,2,0,1,0.347385,38000
2,142019,1,54,2,15652,10,1.32,2,2,0,1,0.349705,38000
3,146816,0,52,2,16560,10,1.10,1,1,0,1,0.359232,28000
4,27699,0,63,5,9078,18,1.10,1,4,0,1,0.529368,30000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114394,119880,1,29,1,29678,10,1.60,2,1,0,1,0.721156,25000
114395,103695,1,29,1,23068,15,1.10,2,3,0,1,0.566393,30000
114396,131933,0,45,1,16418,10,1.10,1,1,0,3,0.327380,30400
114397,146868,0,49,1,12169,10,1.40,1,1,0,1,0.467395,16000


In [ ]:
X_test.loc[((X_test.age >= 18) & (X_test.age <= 20)),  'age'] = 1
X_test.loc[((X_test.age >= 21) & (X_test.age <= 23)),  'age'] = 2
X_test.loc[((X_test.age >= 24) & (X_test.age <= 26)),  'age'] = 3
X_test.loc[((X_test.age >= 27) & (X_test.age <= 30)),  'age'] = 4
X_test.loc[((X_test.age >= 31) & (X_test.age <= 34)),  'age'] = 5
X_test.loc[((X_test.age >= 35) & (X_test.age <= 37)),  'age'] = 6
X_test.loc[((X_test.age >= 38) & (X_test.age <= 42)),  'age'] = 7
X_test.loc[((X_test.age >= 43) & (X_test.age <= 44)),  'age'] = 8
X_test.loc[((X_test.age >= 45) & (X_test.age <= 46)),  'age'] = 9
X_test.loc[((X_test.age >= 47) & (X_test.age <= 50)),  'age'] = 10
X_test.loc[((X_test.age >= 51) & (X_test.age <= 52)),  'age'] = 11
X_test.loc[((X_test.age >= 53) & (X_test.age <= 54)),  'age'] = 12
X_test.loc[((X_test.age >= 55) & (X_test.age <= 56)),  'age'] = 13
X_test.loc[((X_test.age >= 57) & (X_test.age <= 59)),  'age'] = 14
X_test.loc[((X_test.age >= 60) & (X_test.age <= 63)),  'age'] = 15
X_test.loc[((X_test.age >= 64) & (X_test.age <= 66)),  'age'] = 16
X_test.loc[((X_test.age >= 67) & (X_test.age <= 70)),  'age'] = 17
X_test.loc[((X_test.age >= 71) & (X_test.age <= 71)),  'age'] = 18

In [ ]:
test_scaler = StandardScaler().fit(X_test)
# scaler
X_test_scaled = test_scaler.transform(X_test)
X_test_scaled

array([[-0.96746596, -1.59629004,  0.37814137, ..., -0.40100365,
        -0.39327902, -0.40729547],
       [-0.96746596,  2.5767847 ,  3.70495491, ..., -1.28607044,
        -1.20713226, -0.75811575],
       [ 1.0336281 ,  0.19217056, -0.73079648, ..., -0.40100365,
        -1.16191819,  4.40395407],
       ...,
       [ 1.0336281 ,  1.6825544 , -0.73079648, ..., -1.28607044,
         0.42057421, -0.54762359],
       [-0.96746596, -0.70205974,  0.37814137, ..., -1.04468859,
        -0.89063378, -0.0865455 ],
       [-0.96746596,  0.7883241 ,  0.37814137, ..., -0.1596218 ,
         1.23442744, -0.64785795]])

In [ ]:
X_test_minmax = min_max_scaler.fit_transform(X_test)
X_test_minmax

array([[0.        , 0.        , 0.25      , ..., 0.40785614, 0.04663212,
        0.03333441],
       [0.        , 0.82352941, 1.        , ..., 0.29962387, 0.        ,
        0.0236122 ],
       [1.        , 0.35294118, 0.        , ..., 0.40531465, 0.00259067,
        0.16666759],
       ...,
       [1.        , 0.64705882, 0.        , ..., 0.29976218, 0.09326425,
        0.02944552],
       [0.        , 0.17647059, 0.25      , ..., 0.3256189 , 0.01813472,
        0.04222329],
       [0.        , 0.47058824, 0.25      , ..., 0.43477518, 0.13989637,
        0.02666775]])

Предскажем значения для тестовой выборки

In [ ]:
X_lognormal_test = pt.fit_transform(X_test_minmax)

In [ ]:
y_pred = clf.predict(X_test_scaled)

In [ ]:
print("ssssssssssssssssssssssssssssssssssssssssssssssssss")

ssssssssssssssssssssssssssssssssssssssssssssssssss


In [ ]:
y_pred

array([False,  True, False, ..., False, False, False])

Создадим отдельный датасет для отправки на kaggle. 

Данный датасет будет содержать только два необходимых столбца — id и наш ответ.

In [ ]:
df_test = pd.read_csv('credit_test.csv')
df_test.head()

,id,gender,age,marital_status,credit_sum,credit_month,tariff_id,education,credit_count,overdue_credit_count,job_pos,credit_score,region,income
0,39162,F,18,UNM,"39469,00",6,1.44,GRD,NaN,NaN,SPC,"0,424628",САНКТ-ПЕТЕРБУРГ Г,30000.0
1,10673,F,63,WID,"16693,00",10,1.32,GRD,2.0,0.0,SPC,"0,311945",ОБЛ ЛЕНИНГРАДСКАЯ,21250.0
2,121409,M,42,MAR,"60260,00",10,1.10,GRD,5.0,0.0,SPC,"0,421982",МОСКВА Г,150000.0
3,23504,M,41,MAR,"32479,00",12,1.10,GRD,1.0,0.0,DIR,"0,406572",ТАТАРСТАН РЕСП,150000.0
4,69269,F,24,UNM,"8159,00",12,1.10,SCH,2.0,0.0,SPC,"0,418243",ТВЕРСКАЯ ОБЛ,16000.0


In [ ]:
df_kaggle = df_test[['id',]]
df_kaggle['opened'] = y_pred.astype(bool)
df_kaggle.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,opened
0,39162,False
1,10673,True
2,121409,False
3,23504,False
4,69269,True


Сохраним полученный результат в .csv файл для отправки на kaggle

In [ ]:
df_kaggle.to_csv('my_submission.csv', index=False)